In [151]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage import filters
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt, make_path_filter
import geopandas as gpd
import rasterio as rio
import rasterio.mask

In [137]:
# Init Sentinel API
username = 'cryovision' 
password = 'baw.YFV8rfa8nva9mfg' 
api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')

In [138]:
SHAPEFILE = 'pingo_distribution_siberia_ggrosse_bjones/pingo_distribution_siberia_ggrosse_bjones.shp'

In [139]:
# Import Shapefile

cutoff = 2
pingo = gpd.read_file(SHAPEFILE)
pingo = pingo.to_crs('epsg:4326')
pingo = pingo.iloc[0:cutoff]
pingo.to_file('pingo.geojson', driver='GeoJSON')
footprint = geojson_to_wkt(read_geojson('pingo.geojson'))

In [141]:
# Get Images from api

images = api.query(footprint,
                        platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10),
                     limit=10)
dataframe = api.to_geodataframe(images)
dataframe_sorted = dataframe.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True]).head(5)

In [176]:
# # Download Image Data set using Id
path_filter = make_path_filter("*_B0[2]_60m.jp2")
download_data = api.download(dataframe_sorted.index[0],nodefilter=path_filter)

In [177]:
R10 = 'S2B_MSIL2A_20220311T232629_N0400_R130_T01WFP_20220312T000759.SAFE/GRANULE/L2A_T01WFP_A026182_20220311T232631/IMG_DATA/R60m'
b4 = rio.open(R10+'/T01WFP_20220311T232629_B04_60m.jp2')
b3 = rio.open(R10+'/T01WFP_20220311T232629_B03_60m.jp2')
b2 = rio.open(R10+'/T01WFP_20220311T232629_B02_60m.jp2')